# Import Data and Library

In [1]:
import os
os.chdir(r'C:\Users\nadda\Desktop\KU\01204314-65-Statistics-for-Computer-Engineering-Applications\final_project\part2.2')
os.getcwd()

'C:\\Users\\nadda\\Desktop\\KU\\01204314-65-Statistics-for-Computer-Engineering-Applications\\final_project\\part2.2'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import random

In [3]:
orig_df = pd.read_csv('worldbank_gender_2021.csv')
fig_folder = ''
orig_df

,Country Name,Year,Agricultural irrigated land (% of total agricultural land),Agricultural land (% of land area),"Agriculture, forestry, and fishing, value added (% of GDP)",Arable land (% of land area),Cereal yield (kg per hectare),Crop production index (2014-2016 = 100),Fertilizer consumption (kilograms per hectare of arable land),Food production index (2014-2016 = 100),...,Merchandise imports (current US$),Merchandise trade (% of GDP),"Contributing family workers, female (% of female employment) (modeled ILO estimate)","Contributing family workers, male (% of male employment) (modeled ILO estimate)",Firms with female participation in ownership (% of firms),Firms with female top manager (% of firms),"Proportion of time spent on unpaid domestic and care work, female (% of 24 hour day)","Proportion of time spent on unpaid domestic and care work, male (% of 24 hour day)","Women making their own informed decisions regarding sexual relations, contraceptive use and reproductive health care (% of women age 15-49)",Gender Ratio Class
0,Afghanistan,2021,6.51,58.74,33.60,12.00,2099.0,125.67,5.20,114.32,...,5.308000e+09,43.16,60.87,12.79,NaN,NaN,NaN,NaN,NaN,1.0
1,Albania,2021,16.54,41.47,18.36,21.89,5144.7,114.68,NaN,104.92,...,7.718000e+09,62.89,27.58,15.96,NaN,NaN,NaN,NaN,NaN,1.0
2,Algeria,2021,NaN,17.35,11.23,3.16,1433.7,108.76,20.68,104.83,...,3.746600e+10,40.86,2.10,1.61,NaN,NaN,NaN,NaN,NaN,1.0
3,American Samoa,2021,NaN,14.50,NaN,4.85,NaN,NaN,NaN,NaN,...,8.840000e+08,159.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,2021,NaN,39.91,0.53,1.59,NaN,NaN,NaN,NaN,...,1.563000e+09,51.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Virgin Islands (U.S.),2021,NaN,9.43,NaN,2.57,NaN,NaN,NaN,NaN,...,NaN,NaN,0.34,0.21,NaN,NaN,NaN,NaN,NaN,5.0
212,West Bank and Gaza,2021,NaN,64.93,6.32,6.95,2251.9,117.31,371.97,117.39,...,NaN,NaN,8.55,3.44,NaN,NaN,NaN,NaN,NaN,5.0
213,"Yemen, Rep.",2021,NaN,44.42,NaN,2.19,1524.8,124.71,10.63,120.77,...,5.054000e+09,29.68,37.64,9.04,NaN,NaN,NaN,NaN,NaN,1.0
214,Zambia,2021,NaN,32.07,3.00,5.11,2525.0,136.31,63.90,127.49,...,6.435000e+09,74.84,4.87,2.36,NaN,NaN,NaN,NaN,NaN,5.0


In [4]:
df=orig_df.copy()
temp_df=df.drop(['Country Name'], axis=1)
temp_df=df.drop(['Year'], axis=1) # all year is 2021
temp_df = temp_df.dropna(axis=1, how='all') # drop columns where all values are NaN

dummy_df=pd.get_dummies(temp_df)
dummy_df.columns

Index(['Agricultural irrigated land (% of total agricultural land)',
       'Agricultural land (% of land area)',
       'Agriculture, forestry, and fishing, value added (% of GDP)',
       'Arable land (% of land area)', 'Cereal yield (kg per hectare)',
       'Crop production index (2014-2016 = 100)',
       'Fertilizer consumption (kilograms per hectare of arable land)',
       'Food production index (2014-2016 = 100)',
       'Forest area (% of land area)', 'Forest area (sq. km)',
       ...
       'Country Name_Uruguay', 'Country Name_Uzbekistan',
       'Country Name_Vanuatu', 'Country Name_Venezuela, RB',
       'Country Name_Viet Nam', 'Country Name_Virgin Islands (U.S.)',
       'Country Name_West Bank and Gaza', 'Country Name_Yemen, Rep.',
       'Country Name_Zambia', 'Country Name_Zimbabwe'],
      dtype='object', length=365)

In [5]:
cols = set(dummy_df.columns)
cols

{'Access to electricity (% of population)',
 'Adjusted net savings, including particulate emission damage (% of GNI)',
 'Age dependency ratio (% of working-age population)',
 'Agricultural irrigated land (% of total agricultural land)',
 'Agricultural land (% of land area)',
 'Agriculture, forestry, and fishing, value added (% of GDP)',
 'Air transport, registered carrier departures worldwide',
 'Annual freshwater withdrawals, total (% of internal resources)',
 'Arable land (% of land area)',
 'Birth rate, crude (per 1,000 people)',
 'Births attended by skilled health staff (% of total)',
 'CPIA economic management cluster average (1=low to 6=high)',
 'CPIA policies for social inclusion/equity cluster average (1=low to 6=high)',
 'CPIA public sector management and institutions cluster average (1=low to 6=high)',
 'CPIA structural policies cluster average (1=low to 6=high)',
 'Central government debt, total (% of GDP)',
 'Cereal yield (kg per hectare)',
 'Children out of school, primary

In [6]:
all_vars = [
    'Access to electricity (% of population)',
 'Adjusted net savings, including particulate emission damage (% of GNI)',
 'Age dependency ratio (% of working-age population)',
 'Agricultural irrigated land (% of total agricultural land)',
 'Agricultural land (% of land area)',
 'Agriculture, forestry, and fishing, value added (% of GDP)',
 'Air transport, registered carrier departures worldwide',
 'Annual freshwater withdrawals, total (% of internal resources)',
 'Arable land (% of land area)',
 'Birth rate, crude (per 1,000 people)',
 'Births attended by skilled health staff (% of total)',
 'CPIA economic management cluster average (1=low to 6=high)',
 'CPIA policies for social inclusion/equity cluster average (1=low to 6=high)',
 'CPIA public sector management and institutions cluster average (1=low to 6=high)',
 'CPIA structural policies cluster average (1=low to 6=high)',
 'Central government debt, total (% of GDP)',
 'Cereal yield (kg per hectare)',
 'Children out of school, primary, female',
 'Children out of school, primary, male',
 'Completeness of birth registration (%)',
 'Container port traffic (TEU: 20 foot equivalent units)',
 'Contraceptive prevalence, any method (% of married women ages 15-49)',
 'Contributing family workers, female (% of female employment) (modeled ILO estimate)',
 'Contributing family workers, male (% of male employment) (modeled ILO estimate)',
 'Crop production index (2014-2016 = 100)',
 'Current account balance (BoP, current US$)',
 'Death rate, crude (per 1,000 people)',
 'Diabetes prevalence (% of population ages 20 to 79)',
 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)',
 'Expense (% of GDP)',
 'Exports of goods and services (% of GDP)',
 'External debt stocks (% of GNI)',
 'External debt stocks, total (DOD, current US$)',
 'Fertility rate, total (births per woman)',
 'Fertilizer consumption (kilograms per hectare of arable land)',
 'Firms with female participation in ownership (% of firms)',
 'Firms with female top manager (% of firms)',
 'Fixed broadband subscriptions (per 100 people)',
 'Fixed telephone subscriptions (per 100 people)',
 'Food production index (2014-2016 = 100)',
 'Forest area (% of land area)',
 'Forest area (sq. km)',
 'Fuel exports (% of merchandise exports)',
 'GDP (current US$)',
 'GDP growth (annual %)',
 'GNI per capita, PPP (current international $)',
 'GNI, PPP (current international $)',
 'Gender Ratio Class',
 'Government expenditure on education, total (% of GDP)',
 'Grants, excluding technical cooperation (BoP, current US$)',
 'Gross savings (% of GDP)',
 'High-technology exports (% of manufactured exports)',
 'High-technology exports (current US$)',
 'Hospital beds (per 1,000 people)',
 'IDA resource allocation index (1=low to 6=high)',
 'Immunization, DPT (% of children ages 12-23 months)',
 'Immunization, measles (% of children ages 12-23 months)',
 'Imports of goods and services (% of GDP)',
 'Incidence of tuberculosis (per 100,000 people)',
 'Industry (including construction), value added (% of GDP)',
 'Inflation, consumer prices (annual %)',
 'Intentional homicides (per 100,000 people)',
 'Investment in energy with private participation (current US$)',
 'Investment in transport with private participation (current US$)',
 'Investment in water and sanitation with private participation (current US$)',
 'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)',
 'Labor force participation rate, male (% of male population ages 15+) (modeled ILO estimate)',
 'Labor force, female (% of total labor force)',
 'Labor force, total',
 'Land area (sq. km)',
 'Life expectancy at birth, female (years)',
 'Life expectancy at birth, male (years)',
 'Life expectancy at birth, total (years)',
 'Literacy rate, adult female (% of females ages 15 and above)',
 'Literacy rate, adult male (% of males ages 15 and above)',
 'Literacy rate, adult total (% of people ages 15 and above)',
 'Literacy rate, youth female (% of females ages 15-24)',
 'Literacy rate, youth male (% of males ages 15-24)',
 'Literacy rate, youth total (% of people ages 15-24)',
 'Livestock production index (2014-2016 = 100)',
 'Medium and high-tech exports (% manufactured exports)',
 'Medium and high-tech manufacturing value added (% manufacturing value added)',
 'Merchandise exports (current US$)',
 'Merchandise imports (current US$)',
 'Merchandise trade (% of GDP)',
 'Military expenditure (% of GDP)',
 'Mobile cellular subscriptions (per 100 people)',
 'Mortality rate, under-5 (per 1,000 live births)',
 'Net ODA received (% of GNI)',
 'Net ODA received per capita (current US$)',
 'Net acquisition of financial assets (% of GDP)',
 'Net incurrence of liabilities, total (% of GDP)',
 'Net lending (+) / net borrowing (-) (% of GDP)',
 'Net migration',
 'Net official development assistance received (current US$)',
 'Number of surgical procedures (per 100,000 population)',
 'Permanent cropland (% of land area)',
 'Persistence to last grade of primary, female (% of cohort)',
 'Persistence to last grade of primary, male (% of cohort)',
 'Population ages 0-14 (% of total population)',
 'Population ages 15-64 (% of total population)',
 'Population ages 65 and above (% of total population)',
 'Population growth (annual %)',
 'Population in urban agglomerations of more than 1 million (% of total population)',
 'Population, female (% of total population)',
 'Population, total',
 'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)',
 'Pregnant women receiving prenatal care (%)',
 'Prevalence of HIV, total (% of population ages 15-49)',
 'Prevalence of overweight, weight for height (% of children under 5)',
 'Prevalence of severe wasting, weight for height (% of children under 5)',
 'Prevalence of stunting, height for age (% of children under 5)',
 'Prevalence of undernourishment (% of population)',
 'Prevalence of underweight, weight for age (% of children under 5)',
 'Prevalence of wasting, weight for height (% of children under 5)',
 'Price level ratio of PPP conversion factor (GDP) to market exchange rate',
 'Primary completion rate, total (% of relevant age group)',
 'Proportion of seats held by women in national parliaments (%)',
 'Proportion of time spent on unpaid domestic and care work, female (% of 24 hour day)',
 'Proportion of time spent on unpaid domestic and care work, male (% of 24 hour day)',
 'Rail lines (total route-km)',
 'Renewable energy consumption (% of total final energy consumption)',
 'Renewable internal freshwater resources, total (billion cubic meters)',
 'Revenue, excluding grants (% of GDP)',
 'Risk of catastrophic expenditure for surgical care (% of people at risk)',
 'Risk of impoverishing expenditure for surgical care (% of people at risk)',
 'Rural population',
 'Rural population (% of total population)',
 'School enrollment, preprimary (% gross)',
 'School enrollment, primary (% gross)',
 'School enrollment, primary (gross), gender parity index (GPI)',
 'School enrollment, primary and secondary (gross), gender parity index (GPI)',
 'School enrollment, secondary (% gross)',
 'School enrollment, tertiary (% gross)',
 'Secure Internet servers (per 1 million people)',
 'Short-term debt (% of total reserves)',
 'Surface area (sq. km)',
 'Technical cooperation grants (BoP, current US$)',
 'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)',
 'Total debt service (% of exports of goods, services and primary income)',
 'Total reserves (includes gold, current US$)',
 'Trained teachers in primary education (% of total teachers)',
 'Unemployment, total (% of total labor force) (modeled ILO estimate)',
 'Unmet need for contraception (% of married women ages 15-49)',
 'Urban population',
 'Urban population (% of total population)',
 'Vulnerable employment, female (% of female employment) (modeled ILO estimate)',
 'Vulnerable employment, male (% of male employment) (modeled ILO estimate)',
 'Women making their own informed decisions regarding sexual relations, contraceptive use and reproductive health care (% of women age 15-49)'
]

input_vars = all_vars.copy()
output_var = 'Gender Ratio Class'
df[output_var] = df[output_var].apply(lambda x: int(x) if pd.notna(x) else np.nan)
df[output_var] = df[output_var].astype("category")
input_vars.remove(output_var)
input_vars

['Access to electricity (% of population)',
 'Adjusted net savings, including particulate emission damage (% of GNI)',
 'Age dependency ratio (% of working-age population)',
 'Agricultural irrigated land (% of total agricultural land)',
 'Agricultural land (% of land area)',
 'Agriculture, forestry, and fishing, value added (% of GDP)',
 'Air transport, registered carrier departures worldwide',
 'Annual freshwater withdrawals, total (% of internal resources)',
 'Arable land (% of land area)',
 'Birth rate, crude (per 1,000 people)',
 'Births attended by skilled health staff (% of total)',
 'CPIA economic management cluster average (1=low to 6=high)',
 'CPIA policies for social inclusion/equity cluster average (1=low to 6=high)',
 'CPIA public sector management and institutions cluster average (1=low to 6=high)',
 'CPIA structural policies cluster average (1=low to 6=high)',
 'Central government debt, total (% of GDP)',
 'Cereal yield (kg per hectare)',
 'Children out of school, primary

# Cleaning & Outlier Removing

In [7]:
np.random.seed(42)

## Filter Missing Value (Threshold = 60%)

### Before

In [8]:
df.to_csv("before_filter.csv", index=False)

In [9]:
# Use heatmap to visualize missing value (null) positions
df=dummy_df.copy()
#sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
#plt.show()

In [10]:
# Find missing value percent for each variable
null_percent = df[input_vars].isnull().sum()/len(df)*100
null_percent

Access to electricity (% of population)                                                                                                         0.925926
Adjusted net savings, including particulate emission damage (% of GNI)                                                                         45.370370
Age dependency ratio (% of working-age population)                                                                                              0.000000
Agricultural irrigated land (% of total agricultural land)                                                                                     81.018519
Agricultural land (% of land area)                                                                                                              3.240741
                                                                                                                                                 ...    
Urban population                                                                  

In [11]:
for x in input_vars:
    df[x] = df[x].fillna(df[x].median())

df.dropna(subset=[output_var], inplace=True)
df[output_var] = df[output_var].astype(int)

In [12]:
# After replacing missing values, re-check %missing data
df[input_vars].isnull().sum()/len(df)*100

Access to electricity (% of population)                                                                                                        0.0
Adjusted net savings, including particulate emission damage (% of GNI)                                                                         0.0
Age dependency ratio (% of working-age population)                                                                                             0.0
Agricultural irrigated land (% of total agricultural land)                                                                                     0.0
Agricultural land (% of land area)                                                                                                             0.0
                                                                                                                                              ... 
Urban population                                                                                                      

In [13]:
X = df[input_vars]
y = df[output_var]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

# สมมุติว่ามี X, y แล้ว
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Accuracy (macro)
acc_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

f1_macro = make_scorer(f1_score, average='macro')
f1_scores = cross_val_score(model, X, y, cv=5, scoring=f1_macro)

# แสดงผล
print(f"✅ Logistic Regression Classification Before Remove Missing Values Columns")
print("🎯 Accuracy (macro) avg:", acc_scores.mean())
print("🎯 F1 Score (macro) avg:", f1_scores.mean())

✅ Logistic Regression Classification Before Remove Missing Values Columns
🎯 Accuracy (macro) avg: 0.31237553342816504
🎯 F1 Score (macro) avg: 0.2805315425315425


### Filter

In [14]:
df = dummy_df.copy()
# Find missing value percent for each variable
null_percent = df[input_vars].isnull().sum()/len(df)*100
null_percent

Access to electricity (% of population)                                                                                                         0.925926
Adjusted net savings, including particulate emission damage (% of GNI)                                                                         45.370370
Age dependency ratio (% of working-age population)                                                                                              0.000000
Agricultural irrigated land (% of total agricultural land)                                                                                     81.018519
Agricultural land (% of land area)                                                                                                              3.240741
                                                                                                                                                 ...    
Urban population                                                                  

In [15]:
acceptable_missing_ratio = 60
updated_vars = []
for i in range(0,len(input_vars)):  
    # if %missing value of this feature is less than or at least threshold, keep this feature
    if null_percent.iloc[i] <= acceptable_missing_ratio :          
        updated_vars.append(input_vars[i])

input_vars = updated_vars.copy()
all_vars = input_vars.copy()
all_vars.append(output_var)
df = df[all_vars].copy()
dummy_df = df.copy()

In [16]:
null_percent = df.isnull().sum().sort_values(ascending=False)/len(df)*100
null_percent

Trained teachers in primary education (% of total teachers)    55.555556
Short-term debt (% of total reserves)                          54.629630
Intentional homicides (per 100,000 people)                     51.851852
Container port traffic (TEU: 20 foot equivalent units)         51.851852
Children out of school, primary, female                        50.925926
                                                                 ...    
Population ages 15-64 (% of total population)                   0.000000
Population ages 65 and above (% of total population)            0.000000
Population growth (annual %)                                    0.000000
Age dependency ratio (% of working-age population)              0.000000
Population, total                                               0.000000
Length: 108, dtype: float64

### After

In [17]:
df.to_csv(f"after_filter_{acceptable_missing_ratio}_percent.csv", index=False)

In [18]:
# Find missing value percent for each variable
null_percent = df.isnull().sum()/len(df)*100
null_percent

Access to electricity (% of population)                                           0.925926
Adjusted net savings, including particulate emission damage (% of GNI)           45.370370
Age dependency ratio (% of working-age population)                                0.000000
Agricultural land (% of land area)                                                3.240741
Agriculture, forestry, and fishing, value added (% of GDP)                       11.574074
                                                                                   ...    
Urban population                                                                  0.925926
Urban population (% of total population)                                          0.925926
Vulnerable employment, female (% of female employment) (modeled ILO estimate)    13.888889
Vulnerable employment, male (% of male employment) (modeled ILO estimate)        13.888889
Gender Ratio Class                                                               13.888889

In [19]:
for x in input_vars:
    df[x] = df[x].fillna(df[x].median())

df.dropna(subset=[output_var], inplace=True)
df[output_var] = df[output_var].astype(int)

In [20]:
# After replacing missing values, re-check %missing data
df.isnull().sum()/len(df)*100

Access to electricity (% of population)                                          0.0
Adjusted net savings, including particulate emission damage (% of GNI)           0.0
Age dependency ratio (% of working-age population)                               0.0
Agricultural land (% of land area)                                               0.0
Agriculture, forestry, and fishing, value added (% of GDP)                       0.0
                                                                                ... 
Urban population                                                                 0.0
Urban population (% of total population)                                         0.0
Vulnerable employment, female (% of female employment) (modeled ILO estimate)    0.0
Vulnerable employment, male (% of male employment) (modeled ILO estimate)        0.0
Gender Ratio Class                                                               0.0
Length: 108, dtype: float64

In [21]:
X = df[input_vars]
y = df[output_var]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

# สมมุติว่ามี X, y แล้ว
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Accuracy (macro)
acc_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

f1_macro = make_scorer(f1_score, average='macro')
f1_scores = cross_val_score(model, X, y, cv=5, scoring=f1_macro)

# แสดงผล
print(f"✅ Logistic Regression Classification After Remove Missing Values Columns (Threshold = {acceptable_missing_ratio}%)")
print("🎯 Accuracy (macro) avg:", acc_scores.mean())
print("🎯 F1 Score (macro) avg:", f1_scores.mean())

✅ Logistic Regression Classification After Remove Missing Values Columns (Threshold = 60%)
🎯 Accuracy (macro) avg: 0.2849217638691323
🎯 F1 Score (macro) avg: 0.26847206010363905


## Remove Outlier

In [22]:
#dummy_df.isnull().sum().sort_values(ascending=False)/len(dummy_df)*100
dummy_df.isnull().sum()/len(dummy_df)*100

Access to electricity (% of population)                                           0.925926
Adjusted net savings, including particulate emission damage (% of GNI)           45.370370
Age dependency ratio (% of working-age population)                                0.000000
Agricultural land (% of land area)                                                3.240741
Agriculture, forestry, and fishing, value added (% of GDP)                       11.574074
                                                                                   ...    
Urban population                                                                  0.925926
Urban population (% of total population)                                          0.925926
Vulnerable employment, female (% of female employment) (modeled ILO estimate)    13.888889
Vulnerable employment, male (% of male employment) (modeled ILO estimate)        13.888889
Gender Ratio Class                                                               13.888889

In [23]:
X = df[input_vars]
y = df[output_var]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

# สมมุติว่ามี X, y แล้ว
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Accuracy (macro)
acc_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

f1_macro = make_scorer(f1_score, average='macro')
f1_scores = cross_val_score(model, X, y, cv=5, scoring=f1_macro)

# แสดงผล
print(f"✅ Logistic Regression Classification Before Remove Outliers")
print("🎯 Accuracy (macro) avg:", acc_scores.mean())
print("🎯 F1 Score (macro) avg:", f1_scores.mean())

✅ Logistic Regression Classification Before Remove Outliers
🎯 Accuracy (macro) avg: 0.2849217638691323
🎯 F1 Score (macro) avg: 0.26847206010363905


In [24]:
df = dummy_df.copy()
for col in all_vars:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

df.dropna(subset=[output_var], inplace=True)
df[output_var] = df[output_var].astype(int)

# แทน outlier ด้วย median ทีละคอลัมน์
for col in input_vars:
    if pd.api.types.is_numeric_dtype(df[col]):
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        median = df[col].median()

        # แทนค่าที่เป็น outlier ด้วย median
        df.loc[(df[col] < lower) | (df[col] > upper), col] = median

In [25]:
outlier_removed_df = df.copy()

In [26]:
df.isnull().sum().sort_values(ascending=False)/len(df)*100

Access to electricity (% of population)                                              0.0
Net incurrence of liabilities, total (% of GDP)                                      0.0
Population, total                                                                    0.0
Population, female (% of total population)                                           0.0
Population in urban agglomerations of more than 1 million (% of total population)    0.0
                                                                                    ... 
GDP (current US$)                                                                    0.0
Fuel exports (% of merchandise exports)                                              0.0
Forest area (sq. km)                                                                 0.0
Forest area (% of land area)                                                         0.0
Gender Ratio Class                                                                   0.0
Length: 108, dtype: f

In [27]:
X = df[input_vars]
y = df[output_var]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

# สมมุติว่ามี X, y แล้ว
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Accuracy (macro)
acc_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

f1_macro = make_scorer(f1_score, average='macro')
f1_scores = cross_val_score(model, X, y, cv=5, scoring=f1_macro)

# แสดงผล
print(f"✅ Logistic Regression Classification After Remove Outliers")
print("🎯 Accuracy (macro) avg:", acc_scores.mean())
print("🎯 F1 Score (macro) avg:", f1_scores.mean())

✅ Logistic Regression Classification After Remove Outliers
🎯 Accuracy (macro) avg: 0.29143763213530655
🎯 F1 Score (macro) avg: 0.2804251328097876


In [28]:
len(input_vars)

107

In [29]:
dummy_df = df.copy()

In [30]:
df.to_csv("after_remove_outlier.csv", index=False)

# Feature Selection: Recursive Feature Elimination

In [31]:
len(input_vars)

107

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE

# 🔹 เตรียมข้อมูล
X = df.drop(columns=['Gender Ratio Class'])
y = df['Gender Ratio Class']

# 🔹 สร้าง pipeline: Standardize → RFE → LogisticRegression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rfe', RFE(estimator=LogisticRegression(max_iter=1000, class_weight='balanced'), n_features_to_select=20)),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

# 🔹 ตั้งค่าพารามิเตอร์ที่อยาก tune
param_grid = {
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['liblinear'],  # รองรับทั้ง L1, L2
}

# 🔹 ใช้ F1 macro เป็น scoring
grid = GridSearchCV(pipeline, param_grid, scoring=make_scorer(f1_score, average='macro'), cv=5)
grid.fit(X, y)

# 🔹 แสดงผล
print("✅ Best Params:", grid.best_params_)
print("✅ Best F1 Macro:", grid.best_score_)

# 🔹 ฟีเจอร์ที่รอดจาก RFE
rfe_step = grid.best_estimator_.named_steps['rfe']
selected_features = X.columns[rfe_step.support_]

✅ Best Params: {'clf__C': 10, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
✅ Best F1 Macro: 0.3376830588982965


In [33]:
# input_vars = ฟีเจอร์ที่ถูกเลือกจาก RFE
input_vars = selected_features.tolist()

# all_vars = รวม input + output
output_var = 'Gender Ratio Class'
all_vars = input_vars + [output_var]

# กรองข้อมูลเฉพาะคอลัมน์ที่เหลือรอด
df = df[all_vars].copy()

# สำรองไว้ใช้ซ้ำ
dummy_df = df.copy()

In [34]:
X = df[input_vars]
y = df[output_var]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

# สมมุติว่ามี X, y แล้ว
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Accuracy (macro)
acc_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

f1_macro = make_scorer(f1_score, average='macro')
f1_scores = cross_val_score(model, X, y, cv=5, scoring=f1_macro)

# แสดงผล
print(f"✅ Logistic Regression Classification After RFE (From {acceptable_missing_ratio}% Filterd Data)")
print("🎯 Accuracy (macro) avg:", acc_scores.mean())
print("🎯 F1 Score (macro) avg:", f1_scores.mean())

✅ Logistic Regression Classification After RFE (From 60% Filterd Data)
🎯 Accuracy (macro) avg: 0.440169133192389
🎯 F1 Score (macro) avg: 0.42856199854471305


In [35]:
df = df[all_vars]

In [36]:
df.to_csv(f"after_rfe.csv", index=False)

# ML Model

## Hyperparameter Tuning

## Model Evaluation

## Attribute Relationship